In [2]:
import sympy

In [3]:
# QCD
Pv, Pp, Pm, Pqq, Pqg, Pgq, Pgg = sympy.symbols("P_V P_+ P_- P_qq P_qg P_gq P_gg")
# QED
Pxv, Pxp, Pxpu, Pxpd, Pxm, Pxmu, Pxmd, Pxqq, Pxqg, Pxgq, Pxgg = sympy.symbols("P^x_V P^x_+ P^x_+u P^x_+d P^x_- P^x_-u P^x_-d P^x_qq P^x_qg P^x_gq P^x_gg")
Pxqy, Pxuy, Pxdy, Pxyq, Pxyu, Pxyd, Pxyg, Pxgy, Pxyy = sympy.symbols("P^x_q\gamma P^x_u\gamma P^x_d\gamma P^x_\gamma\ q P^x_\gamma\ u P^x_\gamma\ d P^x_\gamma\ g P^x_g\gamma P^x_\gamma\gamma")
eu2, ed2, es2 = sympy.symbols("e_u^2 e_d^2 e_\Sigma^2") # charges
eu4, ed4 = sympy.symbols("e_u^4 e_d^4") # charges

In [4]:
P = {}
ns, s, qed, qcd = "ns", "s", "qed", "qcd"
P[ns, qcd] = sympy.Array.zeros(14,14).as_mutable()
P[ns, qed] = sympy.Array.zeros(14,14).as_mutable()
P[s, qcd] =  sympy.Array.zeros(14,14).as_mutable()
P[s, qed] =  sympy.Array.zeros(14,14).as_mutable()

ei2=[eu2, ed2, ed2, eu2, ed2, eu2]
ei4=[eu4, ed4, ed4, eu4, ed4, eu4]
def es2_(nf):
    nu = int(nf/2)
    nd = nf - nu
    return nu*eu2 + nd*ed2

def es4_(nf):
    nu = int(nf/2)
    nd = nf - nu
    return nu*eu4 + nd*ed4

def P_qcd(nf):
    res = sympy.Matrix.zeros(14,14).as_mutable()
    res[0, 0]=Pgg
    for i in range(1, nf+1):
        res[0, 2*i] = Pgq #g q+
        res[2*i, 0] = 2 * Pqg #q+ g
        res[2*i,2*i] = Pp #q+ q+
        res[1 + 2*i,1 + 2*i] = Pm #q- q-
    return res

def P_qed(nf):
    es2=es2_(nf)
    es4=es4_(nf)
    res = sympy.Matrix.zeros(14,14).as_mutable()
    res[0, 0]=es2*Pxgg
    res[1, 1]=Pxyy #the factor es2 ar O(aem1) and O(as1aem1) and the factor es4 at O(aem2) are inside Pxyy
    res[0, 1]=es2*Pxgy
    res[1, 0]=es2*Pxyg
    for i in [1, 4, 6]:
        if i <= nf:
            res[0, 2*i] = ei2[i-1]*Pxgq
            res[2*i, 0] = 2*ei2[i-1]*Pxqg
            res[1, 2*i] = ei2[i-1]*Pxyu #a factor of eu^2 at O(aem2) is inside Pxyu
            res[2*i, 1] = 2*ei2[i-1]*Pxuy #a factor of eu^2 at O(aem2) is inside Pxuy
            res[2*i,2*i] = ei2[i-1]*Pxpu #a factor of eu^2 at O(aem2) is inside Pxpu
            res[1 + 2*i,1 + 2*i] = ei2[i-1]*Pxmu #a factor of eu^2 at O(aem2) is inside Pxmu
    for i in [2, 3, 5]:
        if i <= nf:
            res[0, 2*i] = ei2[i-1]*Pxgq
            res[2*i, 0] = 2*ei2[i-1]*Pxqg
            res[1, 2*i] = ei2[i-1]*Pxyd #a factor of ed^2 at O(aem2) is inside Pxyd
            res[2*i, 1] = 2*ei2[i-1]*Pxdy #a factor of ed^2 at O(aem2) is inside Pxdy
            res[2*i,2*i] = ei2[i-1]*Pxpd #a factor of ed^2 at O(aem2) is inside Pxpd
            res[1 + 2*i,1 + 2*i] = ei2[i-1]*Pxmd #a factor of ed^2 at O(aem2) is inside Pxmd
    return res

def Ps_qcd(nf):
    res = sympy.Matrix.zeros(14,14).as_mutable()
    for i in range(1, nf+1):
        res[2*i, 2] = Pqq - Pp
        res[2*i, 3] = Pqq - Pp
        res[1 + 2*i, 4] = Pv - Pm
        res[1 + 2*i, 5] = Pv - Pm
    return res/nf

def Ps_qed(nf):
    res = sympy.Matrix.zeros(14,14).as_mutable()
    for i in range(1, nf+1):
        res[2*i, 2] = ei2[i-1]*eu2*(Pxqq - Pxp)
        res[2*i, 3] = ei2[i-1]*ed2*(Pxqq - Pxp)
    return res/nf

def P_uni(nf):
    return P_qcd(nf)+P_qed(nf)

def Ps_uni(nf):
    return Ps_qcd(nf)+Ps_qed(nf)

In [5]:
def rot_fl_to_ev(nf):
    nu = int(nf/2)
    nd = nf - nu
    res = sympy.Matrix.zeros(14,14).as_mutable()
    res[0, 0]=1
    res[1, 1]=1
    for i in range(2 + 2*nf, 14): 
        res[i,i] = 1
    for i in range(1,nf+1): #Sigma and V
        res[2, 2*i] = 1
        res[4, 1 + 2*i] = 1
    for i in [1, 4, 6]:#loop on up quarks
        if i <= nf:
            res[3, 2*i] = nd/nu
            res[5,1 + 2*i] = nd/nu
    for i in [2, 3, 5]:#loop on down quarks
        if i <= nf:
            res[3, 2*i] = -1
            res[5, 1 + 2*i] = -1
    if nf >= 3 :
        res[6, 4] = 1
        res[6, 6] = -1
        res[7, 5] = 1
        res[7, 7] = -1
    if nf >= 4 :
        res[8, 2] = 1
        res[8, 8] = -1
        res[9, 3] = 1
        res[9, 9] = -1
    if nf >= 5 :
        res[10, 4] = 1
        res[10, 6] = 1
        res[10, 10] = -2
        res[11, 5] = 1
        res[11, 7] = 1
        res[11, 11] = -2
    if nf == 6 :
        res[12, 2] = 1
        res[12, 8] = 1
        res[12, 12] = -2
        res[13, 3] = 1
        res[13, 9] = 1
        res[13, 13] = -2
    return res

def rot_ev_to_fl(nf):
    return rot_fl_to_ev(nf).inv()

In [8]:
def rot_sin_to_ev(nf):
    nu = int(nf/2)
    nd = nf - nu
    res = sympy.Matrix.zeros(14,14).as_mutable()
    res[0, 0]=1
    res[1, 1]=1
    res[2,2]=1
    res[2,3]=1
    res[3,2]=nd/nu
    res[3,3]=-1
    res[4,4]=1
    res[4,5]=1
    res[5,4]=nd/nu
    res[5,5]=-1
    for i in range(6,14):
        res[i,i]=1
    return res

def rot_ev_to_sin(nf):
    return rot_sin_to_ev(nf).inv()

In [9]:
def P_ev(nf):
    res = rot_fl_to_ev(nf) * P_uni(nf) * rot_ev_to_fl(nf) + rot_fl_to_ev(nf) * Ps_uni(nf) * rot_ev_to_sin(nf)
    return res

def P_ev_sing(nf):
    return P_ev(nf)[:4,:4]

def P_ev_val(nf):
    return P_ev(nf)[4:6,4:6]

In [11]:
P_ev_sing(6)

Matrix([
[        P^x_gg*(3*e_d^2 + 3*e_u^2) + P_gg,              P^x_g\gamma*(3*e_d^2 + 3*e_u^2),                                                                                                                                    0.5*P^x_gq*e_d^2 + 0.5*P^x_gq*e_u^2 + 1.0*P_gq,                                                                                                                  -0.5*P^x_gq*e_d^2 + 0.5*P^x_gq*e_u^2],
[         P^x_\gamma g*(3*e_d^2 + 3*e_u^2),                             P^x_\gamma\gamma,                                                                                                                                   0.5*P^x_\gamma d*e_d^2 + 0.5*P^x_\gamma u*e_u^2,                                                                                                      -0.5*P^x_\gamma d*e_d^2 + 0.5*P^x_\gamma u*e_u^2],
[6*P^x_qg*e_d^2 + 6*P^x_qg*e_u^2 + 12*P_qg,    6*P^x_d\gamma*e_d^2 + 6*P^x_u\gamma*e_u^2, 0.5*P^x_+d*e_d^2 + 0.5*P^x_+u*e_u^2 - 1.11022302462516e-16*

In [24]:
P_ev_val(6)

Matrix([
[0.5*P^x_-d*e_d^2 + 0.5*P^x_-u*e_u^2 - 1.11022302462516e-16*P_- + 1.0*P_V,          -0.5*P^x_-d*e_d^2 + 0.5*P^x_-u*e_u^2],
[                                    -0.5*P^x_-d*e_d^2 + 0.5*P^x_-u*e_u^2, 0.5*P^x_-d*e_d^2 + 0.5*P^x_-u*e_u^2 + 1.0*P_-]])

In [13]:
def eD2_(nf):
    nu = int(nf/2)
    nd = nf - nu
    return nd*eu2 + nu*ed2
def etam_(nf):
    nu = int(nf/2)
    nd = nf - nu
    return 0.5*(eu2 - ed2)
def P_ev_sing2(nf):
    es2=es2_(nf)
    eD2=eD2_(nf)
    etam=etam_(nf)
    nu = int(nf/2)
    nd = nf - nu
    res = sympy.Matrix([
        [Pgg + es2 * Pxgg, es2 * Pxgy, Pgq + es2/nf*Pxgq, 2*nu/nf*etam*Pxgq],
        [es2 * Pxyg, Pxyy, nu/nf*eu2*Pxyu + nd/nf*ed2*Pxyd, 2*nu/nf*0.5*(eu2*Pxyu-ed2*Pxyd)],
        [2*nf*Pqg + 2*es2*Pxqg, 2*(nu*eu2*Pxuy+nd*ed2*Pxdy), Pqq + (nu*eu2*Pxpu+nd*ed2*Pxpd)/nf +(es2/nf)**2*(Pxqq - Pxp), 2*nu/nf*0.5*(eu2*Pxpu - ed2*Pxpd) +2*nu*etam*es2/nf**2*(Pxqq - Pxp)],
        [4*nd*etam*Pxqg, 4*nd*0.5*(eu2*Pxuy - ed2*Pxdy), 2*nd/nf*0.5*(eu2*Pxpu - ed2*Pxpd) +2*nd*etam*es2/nf**2*(Pxqq - Pxp), Pp + (nd*eu2*Pxpu + nu*ed2*Pxpd)/nf + 4*nu*nd/nf**2*etam**2*(Pxqq - Pxp)]
    ])
    return res

def P_ev_val2(nf):
    nu = int(nf/2)
    nd = nf - nu
    res = sympy.Matrix([
        [Pv+(nu*eu2*Pxmu+nd*ed2*Pxmd)/nf, 2*nu/nf*0.5*(eu2*Pxmu - ed2*Pxmd)],
        [2*nd/nf*0.5*(eu2*Pxmu - ed2*Pxmd), Pm + (nd*eu2*Pxmu + nu*ed2*Pxmd)/nf]
    ])
    return res

In [20]:
sympy.simplify(P_ev_sing(5)-P_ev_sing2(5))

Matrix([
[0, 0,                                                                                                                                                                                                                                8.88178419700125e-17*P^x_gq*e_d^2,                                                                                                                                                            2.77555756156289e-17*P_gq],
[0, 0,                                                                                                                                                                                                                          1.11022302462516e-16*P^x_\gamma d*e_d^2,                                                                                                                                                                                    0],
[0, 0,                                                                                         

In [19]:
sympy.simplify(P_ev_sing(6)-P_ev_sing2(6))

Matrix([
[0, 0, -1.11022302462516e-16*P_gq,                         0],
[0, 0,                          0,                         0],
[0, 0,  -1.11022302462516e-16*P_+,                         0],
[0, 0,                          0, -1.11022302462516e-16*P_+]])

In [15]:
sympy.simplify(P_ev_val(5)-P_ev_val2(5))

Matrix([
[                                                                1.11022302462516e-16*P^x_-d*e_d^2,          2.77555756156289e-17*P_-],
[-1.11022302462516e-16*P^x_-d*e_d^2 + 1.11022302462516e-16*P^x_-u*e_u^2 + 5.55111512312578e-17*P_V, 1.11022302462516e-16*P^x_-u*e_u^2]])

In [16]:
sympy.simplify(P_ev_val(6)-P_ev_val2(6))

Matrix([
[-1.11022302462516e-16*P_-,                         0],
[                        0, -1.11022302462516e-16*P_-]])